# EDA v1 - RentSense

Анализ данных о предложениях аренды недвижимости в Москве.

Цель: понять структуру данных, выявить проблемы и подготовиться к обучению моделей.


In [3]:
# Импорты библиотек
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import dotenv_values
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Библиотеки загружены")
print(f"NumPy: {np.__version__}, Pandas: {pd.__version__}, Matplotlib: {matplotlib.__version__}")


Библиотеки загружены
NumPy: 1.26.4, Pandas: 2.3.2, Matplotlib: 3.9.2


## 1. Подключение к БД и загрузка данных


In [9]:
# Подключение к БД (удаленный сервер 89.110.92.128)
from pathlib import Path
import sys
import subprocess

# Проверка и установка pymysql если нужно
try:
    import pymysql
except ImportError:
    print("Устанавливаю pymysql...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pymysql", "--quiet"])
    import pymysql

env_path = Path('../..') / '.env'
env = dotenv_values(env_path)

DBTYPE = env.get('DB_TYPE') or 'mysql+pymysql'
LOGIN = env.get('DB_LOGIN') or 'root'
PASS = env.get('DB_PASS') or 'rootpassword'
IP = env.get('DB_IP') or '89.110.92.128'
PORT = env.get('DB_PORT') or '3306'
DBNAME = env.get('DB_NAME') or 'rentsense'

# Показываем, какие параметры используются (без пароля)
print(f"Параметры подключения:")
print(f"  IP: {IP}")
print(f"  PORT: {PORT}")
print(f"  USER: {LOGIN}")
print(f"  DB: {DBNAME}")

DATABASE_URL = f'{DBTYPE}://{LOGIN}:{PASS}@{IP}:{PORT}/{DBNAME}?charset=utf8mb4'

print(f"Подключение к БД: {DBNAME}@{IP}:{PORT}")
print(f"Параметры из .env: IP={IP}, PORT={PORT}")

try:
    engine = create_engine(DATABASE_URL, pool_pre_ping=True, connect_args={"connect_timeout": 10})
    # Проверка подключения
    with engine.connect() as conn:
        print("Подключение успешно")
except Exception as e:
    print(f"Ошибка подключения: {e}")
    print("\nПроверьте:")
    print("1. Файл .env с параметрами DB_IP, DB_PORT, DB_LOGIN, DB_PASS")
    print("2. Доступность сервера БД (89.110.92.128:3306)")
    print("3. Правильность учетных данных")
    print("4. Firewall/сеть позволяет подключение к удаленному серверу")
    raise


Параметры подключения:
  IP: 89.110.92.128
  PORT: 3306
  USER: rentsense
  DB: rentsense
Подключение к БД: rentsense@89.110.92.128:3306
Параметры из .env: IP=89.110.92.128, PORT=3306
Подключение успешно


In [ ]:
# Загрузка данных
# Проверка, что engine определен (ячейка 3 должна выполниться успешно)
if 'engine' not in locals() and 'engine' not in globals():
    raise NameError("Переменная 'engine' не определена. Сначала запустите ячейку 3 (подключение к БД)")

query = """
SELECT 
    o.cian_id,
    o.price,
    o.price_changes,
    o.category,
    o.views_count,
    o.photos_count,
    o.floor_number,
    o.floors_count,
    o.publication_at,
    o.created_at as offer_created_at,
    o.updated_at as offer_updated_at,
    
    a.county,
    a.district,
    a.street,
    a.house,
    a.metro,
    a.travel_type,
    a.travel_time,
    a.coordinates,
    
    ri.repair_type,
    ri.total_area,
    ri.living_area,
    ri.kitchen_area,
    ri.ceiling_height,
    ri.balconies,
    ri.loggias,
    ri.rooms_count,
    ri.separated_wc,
    ri.combined_wc,
    ri.windows_view,
    
    ro.build_year,
    ro.entrances,
    ro.material_type,
    ro.parking_type,
    ro.garbage_chute,
    ro.lifts_count,
    ro.passenger_lifts,
    ro.cargo_lifts,
    
    rd.realty_type,
    rd.project_type,
    rd.heat_type,
    rd.gas_type,
    rd.is_apartment,
    rd.is_penthouse,
    rd.is_mortgage_allowed,
    rd.is_premium,
    rd.is_emergency,
    
    od.deal_type,
    od.flat_type,
    od.payment_period,
    od.deposit,
    od.prepay_months,
    od.utilities_included,
    od.client_fee,
    od.agent_fee,
    od.description,
    
    d.name as developer_name,
    d.review_count as developer_review_count,
    d.total_rate as developer_rate,
    d.buildings_count as developer_buildings_count,
    d.foundation_year as developer_foundation_year,
    d.is_reliable as developer_is_reliable
FROM offers o
LEFT JOIN addresses a ON o.cian_id = a.cian_id
LEFT JOIN realty_inside ri ON o.cian_id = ri.cian_id
LEFT JOIN realty_outside ro ON o.cian_id = ro.cian_id
LEFT JOIN realty_details rd ON o.cian_id = rd.cian_id
LEFT JOIN offers_details od ON o.cian_id = od.cian_id
LEFT JOIN developers d ON o.cian_id = d.cian_id
"""

print("Загрузка данных из БД...")
df = pd.read_sql(query, engine)
print(f"Загружено строк: {len(df)}")
df.head()


Загрузка данных из БД...
Загружено строк: 4638


,cian_id,price,category,views_count,photos_count,floor_number,floors_count,publication_at,offer_created_at,offer_updated_at,...,utilities_included,client_fee,agent_fee,description,developer_name,developer_review_count,developer_rate,developer_buildings_count,developer_foundation_year,developer_is_reliable
0,325573108,122400.0,flatRent,None,22.0,3,28,1.767697e+09,2026-01-09 15:35:40,2026-01-09 15:35:40,...,1.0,0.0,0.0,Идут показы с датой заселения после 21.01.2026...,Галс-Девелопмент,194.0,4.7,178.0,1994.0,0.0
1,325353038,1200000.0,flatRent,None,48.0,12,18,1.766627e+09,2026-01-09 15:35:54,2026-01-09 15:35:54,...,1.0,0.0,0.0,Лот 554223. Бонус коллегам! Предлагается в аре...,None,NaN,NaN,NaN,NaN,NaN
2,325047081,225000.0,flatRent,None,11.0,3,5,1.765795e+09,2026-01-09 15:36:41,2026-01-09 15:36:41,...,1.0,0.0,0.0,Просторная квартира с 3 спальнями площадью 100...,None,NaN,NaN,NaN,NaN,NaN
3,325322890,1200000.0,flatRent,None,25.0,12,18,1.766530e+09,2026-01-09 15:46:19,2026-01-09 15:46:19,...,1.0,0.0,0.0,Просторная квартира с 4 спальнями общей площад...,None,NaN,NaN,NaN,NaN,NaN
4,325225143,55000.0,flatRent,None,20.0,3,5,1.766241e+09,2026-01-09 15:50:22,2026-01-09 15:50:22,...,1.0,60.0,50.0,Сдаётся просторная 2-комнатная квартира с евро...,None,NaN,NaN,NaN,NaN,NaN


### 1.1 Анализ price_changes и актуальной цены

Проверяем, актуальная ли цена в `offers.price` или нужно брать из `price_changes`


In [ ]:
# Анализ price_changes - извлечение актуальной цены
import json

def get_latest_price_from_changes(price_changes_json):
    """Извлекает последнюю цену из price_changes по дате changeTime"""
    if not price_changes_json:
        return None
    try:
        if isinstance(price_changes_json, str):
            changes = json.loads(price_changes_json)
        else:
            changes = price_changes_json
            
        if not isinstance(changes, list) or len(changes) == 0:
            return None
        
        # Сортируем по changeTime (последняя дата = актуальная цена)
        sorted_changes = sorted(
            changes, 
            key=lambda x: x.get('changeTime', ''),
            reverse=True
        )
        
        latest = sorted_changes[0]
        return latest.get('priceData', {}).get('price')
    except Exception as e:
        return None

# Применяем функцию для получения актуальной цены из price_changes
df['price_from_changes'] = df['price_changes'].apply(get_latest_price_from_changes)

# Сравнение цен
df['price_diff'] = df['price'] - pd.to_numeric(df['price_from_changes'], errors='coerce')
df['price_matches'] = (abs(df['price_diff']) < 0.01) | (df['price_from_changes'].isna())

print(f"Записей с price_changes: {df['price_changes'].notna().sum()} ({df['price_changes'].notna().sum()/len(df)*100:.1f}%)")
print(f"Записей с актуальной ценой из price_changes: {df['price_from_changes'].notna().sum()} ({df['price_from_changes'].notna().sum()/len(df)*100:.1f}%)")
print(f"\nСравнение цен:")
print(f"  Цены совпадают: {df['price_matches'].sum()} ({df['price_matches'].sum()/len(df)*100:.1f}%)")
print(f"  Цены различаются: {(~df['price_matches']).sum()} ({(~df['price_matches']).sum()/len(df)*100:.1f}%)")

if (~df['price_matches']).sum() > 0:
    print(f"\nПримеры различий (первые 5):")
    diff_examples = df[~df['price_matches']][['cian_id', 'price', 'price_from_changes', 'price_diff']].head()
    print(diff_examples.to_string())

# Решение: использовать price_from_changes если есть, иначе offers.price
df['price_actual'] = df['price_from_changes'].fillna(df['price'])
print(f"\nИспользование актуальной цены:")
print(f"  Из price_changes: {(df['price_actual'] == df['price_from_changes']).sum()}")
print(f"  Из offers.price: {(df['price_actual'] == df['price']).sum()}")


## 2. Общий обзор датасета


In [11]:
print(f"Размер датасета: {df.shape}")
print(f"\nКолонки ({len(df.columns)}):")
print(df.columns.tolist())
print(f"\nТипы данных:")
print(df.dtypes)
print(f"\nОсновная информация:")
df.info()


Размер датасета: (4638, 61)

Колонки (61):
['cian_id', 'price', 'category', 'views_count', 'photos_count', 'floor_number', 'floors_count', 'publication_at', 'offer_created_at', 'offer_updated_at', 'county', 'district', 'street', 'house', 'metro', 'travel_type', 'travel_time', 'coordinates', 'repair_type', 'total_area', 'living_area', 'kitchen_area', 'ceiling_height', 'balconies', 'loggias', 'rooms_count', 'separated_wc', 'combined_wc', 'windows_view', 'build_year', 'entrances', 'material_type', 'parking_type', 'garbage_chute', 'lifts_count', 'passenger_lifts', 'cargo_lifts', 'realty_type', 'project_type', 'heat_type', 'gas_type', 'is_apartment', 'is_penthouse', 'is_mortgage_allowed', 'is_premium', 'is_emergency', 'deal_type', 'flat_type', 'payment_period', 'deposit', 'prepay_months', 'utilities_included', 'client_fee', 'agent_fee', 'description', 'developer_name', 'developer_review_count', 'developer_rate', 'developer_buildings_count', 'developer_foundation_year', 'developer_is_reliabl

In [12]:
# Статистика по числовым признакам
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("Описательная статистика числовых признаков:")
df[numeric_cols].describe()


Описательная статистика числовых признаков:


,cian_id,price,photos_count,floor_number,floors_count,publication_at,travel_time,total_area,living_area,kitchen_area,...,deposit,prepay_months,utilities_included,client_fee,agent_fee,developer_review_count,developer_rate,developer_buildings_count,developer_foundation_year,developer_is_reliable
count,4.638000e+03,4.638000e+03,4619.000000,4638.000000,4638.000000,4.636000e+03,4597.000000,4638.000000,2728.000000,3466.000000,...,4.204000e+03,3605.000000,4585.0,3734.000000,3734.000000,1909.00000,1909.000000,1940.000000,1905.000000,1943.0
mean,2.999920e+08,1.174376e+06,19.500541,8.636266,16.383786,1.725995e+09,9.922993,77.724196,47.166532,12.062983,...,1.894176e+05,1.018863,1.0,28.143278,24.773969,1821.33054,4.176847,3029.817526,1999.330184,0.0
std,6.332391e+07,1.734268e+07,10.165955,8.483215,12.732029,9.166030e+07,5.421501,75.403061,45.992103,8.624247,...,3.873084e+05,0.215079,0.0,29.152014,30.016966,2211.66680,0.835035,4045.628882,9.146431,0.0
min,3.184068e+06,1.500000e+03,2.000000,-2.000000,1.000000,1.255339e+09,1.000000,10.000000,1.000000,1.000000,...,1.000000e+00,1.000000,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,1937.000000,0.0
25%,3.212697e+08,4.900000e+04,12.000000,3.000000,8.000000,1.756382e+09,6.000000,35.200000,19.850000,7.000000,...,4.500000e+04,1.000000,1.0,0.000000,0.000000,133.00000,4.200000,1.500000,1994.000000,0.0
50%,3.256381e+08,7.500000e+04,17.000000,6.000000,14.000000,1.768081e+09,9.000000,50.000000,31.000000,10.000000,...,7.500000e+04,1.000000,1.0,30.000000,0.000000,629.00000,4.400000,956.500000,1994.000000,0.0
75%,3.257445e+08,1.700000e+05,25.000000,11.000000,22.000000,1.768319e+09,13.000000,90.000000,55.000000,15.000000,...,1.800000e+05,1.000000,1.0,50.000000,50.000000,2659.00000,4.500000,4743.000000,2005.000000,0.0
max,3.258169e+08,8.055300e+08,50.000000,76.000000,97.000000,1.768464e+09,26.000000,700.000000,500.000000,150.000000,...,9.000000e+06,11.000000,1.0,100.000000,100.000000,5792.00000,5.000000,19821.000000,2022.000000,0.0


In [ ]:
# Используем актуальную цену для анализа (из price_changes или offers.price)
if 'price_actual' not in df.columns:
    # Если ячейка 6 не выполнилась, используем offers.price
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
else:
    df['price'] = pd.to_numeric(df['price_actual'], errors='coerce')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Гистограмма
axes[0, 0].hist(df['price'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение цен (гистограмма)')
axes[0, 0].set_xlabel('Цена (руб.)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

# Boxplot
axes[0, 1].boxplot(df['price'].dropna(), vert=True)
axes[0, 1].set_title('Распределение цен (boxplot)')
axes[0, 1].set_ylabel('Цена (руб.)')
axes[0, 1].grid(True, alpha=0.3)

# Логарифмированная шкала
log_price = np.log1p(df['price'].dropna())
axes[1, 0].hist(log_price, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Распределение цен (логарифмированная шкала)')
axes[1, 0].set_xlabel('log(Цена + 1)')
axes[1, 0].set_ylabel('Частота')
axes[1, 0].grid(True, alpha=0.3)

# Q-Q plot для проверки нормальности
from scipy import stats
stats.probplot(df['price'].dropna(), dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q plot цены')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Статистика по цене:")
print(df['price'].describe())
print(f"\nМедиана: {df['price'].median():.2f}")
print(f"Среднее: {df['price'].mean():.2f}")
print(f"Мода: {df['price'].mode().values[0] if len(df['price'].mode()) > 0 else 'N/A'}")


Статистика по цене:
count    4.638000e+03
mean     1.174376e+06
std      1.734268e+07
min      1.500000e+03
25%      4.900000e+04
50%      7.500000e+04
75%      1.700000e+05
max      8.055300e+08
Name: price, dtype: float64

Медиана: 75000.00
Среднее: 1174376.35
Мода: 60000.0


### 3.2 Распределение числовых признаков


In [14]:
# Основные числовые признаки для анализа
key_numeric = ['total_area', 'living_area', 'kitchen_area', 'floor_number', 
               'floors_count', 'build_year', 'rooms_count', 'ceiling_height']

# Фильтруем только те колонки, которые есть в датасете
key_numeric = [col for col in key_numeric if col in df.columns]

fig, axes = plt.subplots(len(key_numeric), 2, figsize=(15, 4 * len(key_numeric)))

for i, col in enumerate(key_numeric):
    if col in df.columns:
        data = pd.to_numeric(df[col], errors='coerce').dropna()
        
        if len(data) > 0:
            # Гистограмма
            axes[i, 0].hist(data, bins=30, edgecolor='black', alpha=0.7)
            axes[i, 0].set_title(f'Распределение {col}')
            axes[i, 0].set_xlabel(col)
            axes[i, 0].set_ylabel('Частота')
            axes[i, 0].grid(True, alpha=0.3)
            
            # Boxplot
            axes[i, 1].boxplot(data, vert=True)
            axes[i, 1].set_title(f'Boxplot {col}')
            axes[i, 1].set_ylabel(col)
            axes[i, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 3.3 Распределение категориальных признаков


In [15]:
# Основные категориальные признаки
cat_cols = ['district', 'repair_type', 'material_type', 'realty_type', 
            'deal_type', 'flat_type', 'rooms_count']

# Фильтруем только те колонки, которые есть в датасете
cat_cols = [col for col in cat_cols if col in df.columns]

fig, axes = plt.subplots(len(cat_cols), 1, figsize=(15, 5 * len(cat_cols)))

for i, col in enumerate(cat_cols):
    if col in df.columns:
        value_counts = df[col].value_counts().head(15)  # Топ-15 значений
        
        axes[i].barh(range(len(value_counts)), value_counts.values)
        axes[i].set_yticks(range(len(value_counts)))
        axes[i].set_yticklabels(value_counts.index)
        axes[i].set_title(f'Распределение {col} (топ-15)')
        axes[i].set_xlabel('Количество')
        axes[i].grid(True, alpha=0.3, axis='x')
        
        # Поворачиваем длинные метки
        axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

# Выводим статистику по категориальным признакам
for col in cat_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Уникальных значений: {df[col].nunique()}")
        print(f"  Пропусков: {df[col].isna().sum()} ({df[col].isna().sum() / len(df) * 100:.1f}%)")
        print(f"  Топ-5:")
        print(df[col].value_counts().head())



district:
  Уникальных значений: 125
  Пропусков: 361 (7.8%)
  Топ-5:
district
Пресненский    278
Хамовники      175
Тверской       173
Хорошевский    171
Арбат          145
Name: count, dtype: int64

repair_type:
  Уникальных значений: 4
  Пропусков: 989 (21.3%)
  Топ-5:
repair_type
euro        1623
design      1295
cosmetic     673
no            58
Name: count, dtype: int64

material_type:
  Уникальных значений: 7
  Пропусков: 1229 (26.5%)
  Топ-5:
material_type
none        934
monolith    821
brick       708
panel       703
block       145
Name: count, dtype: int64

realty_type:
  Уникальных значений: 1
  Пропусков: 0 (0.0%)
  Топ-5:
realty_type
flat    4638
Name: count, dtype: int64

deal_type:
  Уникальных значений: 2
  Пропусков: 0 (0.0%)
  Топ-5:
deal_type
rent    4590
sale      48
Name: count, dtype: int64

flat_type:
  Уникальных значений: 3
  Пропусков: 0 (0.0%)
  Топ-5:
flat_type
rooms       3806
studio       768
openPlan      64
Name: count, dtype: int64

rooms_count:
  Ун

## 4. Анализ пропусков


In [16]:
# Подсчет пропусков
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Пропусков': missing,
    'Процент': missing_percent
}).sort_values('Процент', ascending=False)

print("Колонки с пропусками:")
print(missing_df[missing_df['Пропусков'] > 0])

# Визуализация пропусков
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Heatmap пропусков (только колонки с пропусками)
cols_with_missing = missing_df[missing_df['Пропусков'] > 0].index[:30]  # Топ-30
if len(cols_with_missing) > 0:
    sns.heatmap(df[cols_with_missing].isnull(), yticklabels=False, 
                cbar=True, cmap='viridis', ax=axes[0])
    axes[0].set_title('Heatmap пропусков (топ-30 колонок)')
    
    # Bar plot процента пропусков
    top_missing = missing_df.head(20)
    axes[1].barh(range(len(top_missing)), top_missing['Процент'].values)
    axes[1].set_yticks(range(len(top_missing)))
    axes[1].set_yticklabels(top_missing.index, fontsize=8)
    axes[1].set_xlabel('Процент пропусков')
    axes[1].set_title('Топ-20 колонок по проценту пропусков')
    axes[1].grid(True, alpha=0.3, axis='x')
    axes[1].invert_yaxis()

plt.tight_layout()
plt.show()


Колонки с пропусками:
                           Пропусков     Процент
views_count                     4638  100.000000
is_mortgage_allowed             4594   99.051315
is_penthouse                    4098   88.357050
developer_foundation_year       2733   58.926261
developer_rate                  2729   58.840017
developer_review_count          2729   58.840017
developer_buildings_count       2698   58.171626
developer_name                  2698   58.171626
developer_is_reliable           2695   58.106943
garbage_chute                   2607   56.209573
project_type                    2383   51.379905
ceiling_height                  2283   49.223803
living_area                     1910   41.181544
entrances                       1904   41.052178
cargo_lifts                     1809   39.003881
loggias                         1547   33.354894
windows_view                    1404   30.271669
parking_type                    1297   27.964640
balconies                       1278   27.55498

### Стратегии заполнения пропусков

Предварительные рекомендации:
- **Числовые признаки**: медиана или среднее, в зависимости от распределения
- **Категориальные признаки**: мода или "unknown"
- **Координаты**: могут быть критичными для geo-фичей
- **build_year**: можно попробовать восстановить по району/материалу


## 5. Выбросы

### 5.1 IQR метод


In [17]:
def detect_outliers_iqr(df, column):
    """Обнаружение выбросов методом IQR"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Анализ выбросов для ключевых числовых признаков
key_columns = ['price', 'total_area', 'living_area', 'floor_number', 'build_year']

outliers_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0:
            outliers, lower, upper = detect_outliers_iqr(df, col)
            outliers_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'lower_bound': lower,
                'upper_bound': upper,
                'min_value': df[col].min(),
                'max_value': df[col].max()
            }

# Вывод результатов
print("Выбросы (IQR метод):")
for col, stats in outliers_summary.items():
    print(f"\n{col}:")
    print(f"  Выбросов: {stats['count']} ({stats['percent']:.1f}%)")
    print(f"  Границы: [{stats['lower_bound']:.2f}, {stats['upper_bound']:.2f}]")
    print(f"  Диапазон данных: [{stats['min_value']:.2f}, {stats['max_value']:.2f}]")

# Визуализация выбросов
fig, axes = plt.subplots(len(outliers_summary), 1, figsize=(12, 4 * len(outliers_summary)))

for i, (col, stats) in enumerate(outliers_summary.items()):
    data = pd.to_numeric(df[col], errors='coerce').dropna()
    axes[i].boxplot(data, vert=True)
    axes[i].axhline(y=stats['lower_bound'], color='r', linestyle='--', alpha=0.5, label='Lower bound')
    axes[i].axhline(y=stats['upper_bound'], color='r', linestyle='--', alpha=0.5, label='Upper bound')
    axes[i].set_title(f'Выбросы в {col} (IQR метод)')
    axes[i].set_ylabel(col)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


Выбросы (IQR метод):

price:
  Выбросов: 653 (14.1%)
  Границы: [-132500.00, 351500.00]
  Диапазон данных: [1500.00, 805530000.00]

total_area:
  Выбросов: 435 (9.4%)
  Границы: [-47.00, 172.20]
  Диапазон данных: [10.00, 700.00]

living_area:
  Выбросов: 265 (5.7%)
  Границы: [-32.88, 107.73]
  Диапазон данных: [1.00, 500.00]

floor_number:
  Выбросов: 252 (5.4%)
  Границы: [-9.00, 23.00]
  Диапазон данных: [-2.00, 76.00]

build_year:
  Выбросов: 13 (0.3%)
  Границы: [1892.50, 2096.50]
  Диапазон данных: [1785.00, 2025.00]


### 5.2 Z-score метод


In [18]:
def detect_outliers_zscore(df, column, threshold=3):
    """Обнаружение выбросов методом Z-score"""
    df_col = pd.to_numeric(df[column], errors='coerce')
    z_scores = np.abs((df_col - df_col.mean()) / df_col.std())
    outliers = df[z_scores > threshold]
    return outliers, z_scores

# Анализ выбросов Z-score
zscore_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0 and df_col.std() > 0:
            outliers, z_scores = detect_outliers_zscore(df, col, threshold=3)
            zscore_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'max_z_score': z_scores.max() if len(z_scores) > 0 else 0
            }

print("Выбросы (Z-score метод, threshold=3):")
for col, stats in zscore_summary.items():
    print(f"{col}: {stats['count']} выбросов ({stats['percent']:.1f}%), max Z-score: {stats['max_z_score']:.2f}")


Выбросы (Z-score метод, threshold=3):
price: 18 выбросов (0.4%), max Z-score: 46.38
total_area: 111 выбросов (2.4%), max Z-score: 8.25
living_area: 56 выбросов (1.2%), max Z-score: 9.85
floor_number: 82 выбросов (1.8%), max Z-score: 7.94
build_year: 5 выбросов (0.1%), max Z-score: 5.99


### 5.3 Анализ экстремальных значений цены

Проверим самые дорогие и дешевые предложения


In [19]:
# Самые дорогие и дешевые предложения
print("Топ-10 самых дорогих:")
print(df.nlargest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

print("\nТоп-10 самых дешевых:")
print(df.nsmallest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

# Цена за квадратный метр
df['price_per_sqm'] = df['price'] / pd.to_numeric(df['total_area'], errors='coerce')

print("\nСтатистика цены за м²:")
print(df['price_per_sqm'].describe())

# Визуализация цены за м²
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].hist(df['price_per_sqm'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение цены за м²')
axes[0].set_xlabel('Цена за м² (руб.)')
axes[0].set_ylabel('Частота')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(df['price_per_sqm'].dropna(), vert=True)
axes[1].set_title('Boxplot цены за м²')
axes[1].set_ylabel('Цена за м² (руб.)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


Топ-10 самых дорогих:
       cian_id        price  total_area             district  rooms_count repair_type
648  313037687  805530000.0       244.1          Пресненский          2.0        None
639  217791998  482267606.0       685.0  Очаково-Матвеевское          NaN          no
652  309118675  406778840.0       220.0            Хамовники          4.0          no
638  209040594  288586119.0       409.9  Очаково-Матвеевское          6.0          no
630    4349182  236557541.0       224.6          Гагаринский          4.0          no
635  179176213  170000000.0       164.0            Хамовники          5.0        euro
653  324864015  170000000.0       120.0             Якиманка          3.0      design
634  169579402  155000000.0       170.0                Арбат          3.0      design
646  286258299  148230309.0       104.2             Тверской          2.0        None
643  322430288  128304720.0        90.4         Дорогомилово          2.0        None

Топ-10 самых дешевых:
        c

## 6. Утечка по времени

Проверяем, что train и test разделены по времени публикации (test позже train).


In [20]:
# Преобразование publication_at из timestamp в datetime
df['publication_date'] = pd.to_datetime(df['publication_at'], unit='s', errors='coerce')

# Удаляем строки без даты публикации
df_with_date = df.dropna(subset=['publication_date']).copy()

print(f"Всего записей: {len(df)}")
print(f"Записей с датой публикации: {len(df_with_date)}")
print(f"\nДиапазон дат публикации:")
print(f"  От: {df_with_date['publication_date'].min()}")
print(f"  До: {df_with_date['publication_date'].max()}")

# Распределение по времени
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Гистограмма по датам
axes[0].hist(df_with_date['publication_date'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение публикаций по времени')
axes[0].set_xlabel('Дата публикации')
axes[0].set_ylabel('Количество публикаций')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Пример разделения train/test (последние 30 дней = test)
split_date = df_with_date['publication_date'].max() - pd.Timedelta(days=30)
train_mask = df_with_date['publication_date'] <= split_date
test_mask = df_with_date['publication_date'] > split_date

train_data = df_with_date[train_mask]
test_data = df_with_date[test_mask]

print(f"\nРазделение на train/test (split_date = {split_date.date()}):")
print(f"  Train: {len(train_data)} записей ({len(train_data)/len(df_with_date)*100:.1f}%)")
print(f"  Test: {len(test_data)} записей ({len(test_data)/len(df_with_date)*100:.1f}%)")

# Визуализация разделения
axes[1].scatter(train_data['publication_date'], train_data['price'], 
                alpha=0.3, label='Train', s=10)
axes[1].scatter(test_data['publication_date'], test_data['price'], 
                alpha=0.3, label='Test', s=10, color='red')
axes[1].axvline(x=split_date, color='green', linestyle='--', linewidth=2, label='Split date')
axes[1].set_title('Разделение train/test по времени')
axes[1].set_xlabel('Дата публикации')
axes[1].set_ylabel('Цена (руб.)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Проверка, что test позже train
if len(train_data) > 0 and len(test_data) > 0:
    max_train_date = train_data['publication_date'].max()
    min_test_date = test_data['publication_date'].min()
    print(f"\nПроверка временного разделения:")
    print(f"  Max train date: {max_train_date}")
    print(f"  Min test date: {min_test_date}")
    print(f"  Test позже train: {min_test_date > max_train_date} ✓" if min_test_date > max_train_date else "  ⚠️ ПРОБЛЕМА: test содержит более старые данные!")


Всего записей: 4638
Записей с датой публикации: 4636

Диапазон дат публикации:
  От: 2009-10-12 09:11:38
  До: 2026-01-15 08:05:41

Разделение на train/test (split_date = 2025-12-16):
  Train: 1743 записей (37.6%)
  Test: 2893 записей (62.4%)

Проверка временного разделения:
  Max train date: 2025-12-16 08:05:30
  Min test date: 2025-12-16 09:42:01
  Test позже train: True ✓


## 7. Корреляции

Анализ взаимосвязей между признаками


In [21]:
# Выбираем числовые колонки для корреляционного анализа
numeric_for_corr = ['price', 'total_area', 'living_area', 'kitchen_area', 
                    'floor_number', 'floors_count', 'build_year', 'rooms_count',
                    'ceiling_height', 'views_count', 'photos_count']

numeric_for_corr = [col for col in numeric_for_corr if col in df.columns]

# Вычисляем корреляции
corr_matrix = df[numeric_for_corr].apply(pd.to_numeric, errors='coerce').corr()

# Визуализация
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

# Корреляции с ценой
if 'price' in corr_matrix.columns:
    price_corr = corr_matrix['price'].sort_values(ascending=False)
    print("\nКорреляции с ценой:")
    print(price_corr)



Корреляции с ценой:
price             1.000000
total_area        0.126303
kitchen_area      0.084868
ceiling_height    0.083178
living_area       0.074193
rooms_count       0.036917
build_year        0.017032
floor_number      0.007650
photos_count      0.000168
floors_count     -0.009700
views_count            NaN
Name: price, dtype: float64


## 9. Краткий анализ результатов EDA

### Основные находки по каждой ячейке:


In [ ]:
# Итоговый анализ результатов
print("=" * 60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ EDA")
print("=" * 60)

print(f"\n1. РАЗМЕР ДАТАСЕТА:")
print(f"   Всего записей: {len(df)}")
print(f"   Колонок: {len(df.columns)}")

print(f"\n2. ЦЕНЫ (КРИТИЧНО!):")
if 'price_actual' in df.columns:
    price_col = 'price_actual'
else:
    price_col = 'price'
    
price_stats = df[price_col].describe()
print(f"   Медиана: {price_stats['50%']:,.0f} руб")
print(f"   Среднее: {price_stats['mean']:,.0f} руб")
print(f"   Min: {price_stats['min']:,.0f} руб")
print(f"   Max: {price_stats['max']:,.0f} руб")
print(f"   ПРОБЛЕМА: Максимальная цена {price_stats['max']:,.0f} руб - явный выброс!")
print(f"   Std: {price_stats['std']:,.0f} руб (очень большое отклонение)")

# Проверка выбросов
Q1 = price_stats['25%']
Q3 = price_stats['75%']
IQR = Q3 - Q1
outliers_count = len(df[(df[price_col] < Q1 - 1.5*IQR) | (df[price_col] > Q3 + 1.5*IQR)])
print(f"   Выбросов (IQR): {outliers_count} ({outliers_count/len(df)*100:.1f}%)")

print(f"\n3. ПРОПУСКИ (топ-10 проблемных колонок):")
missing_top10 = df.isnull().sum().sort_values(ascending=False).head(10)
for col, count in missing_top10.items():
    print(f"   {col}: {count} ({count/len(df)*100:.1f}%)")

print(f"\n4. КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ:")
print(f"   deal_type:")
if 'deal_type' in df.columns:
    print(f"      {df['deal_type'].value_counts().to_dict()}")
    if 'sale' in df['deal_type'].values:
        print(f"      ВНИМАНИЕ: {len(df[df['deal_type'] == 'sale'])} записей с sale - нужно фильтровать!")

print(f"\n5. ВРЕМЕННОЕ РАЗДЕЛЕНИЕ:")
if 'publication_date' in df.columns:
    df_with_date = df.dropna(subset=['publication_date'])
    print(f"   Диапазон: {df_with_date['publication_date'].min()} - {df_with_date['publication_date'].max()}")
    split_date = df_with_date['publication_date'].max() - pd.Timedelta(days=30)
    train_size = len(df_with_date[df_with_date['publication_date'] <= split_date])
    test_size = len(df_with_date[df_with_date['publication_date'] > split_date])
    print(f"   Train/Test (30 дней): {train_size}/{test_size}")

print(f"\n6. КРИТИЧЕСКИЕ ПРОБЛЕМЫ:")
issues = []
if price_stats['max'] > 100000000:  # > 100 млн
    issues.append(f"Выбросы в цене: max={price_stats['max']:,.0f} руб")
if 'deal_type' in df.columns and (df['deal_type'] == 'sale').any():
    issues.append(f"Записи с deal_type='sale': {len(df[df['deal_type'] == 'sale'])} шт")
if df['total_area'].isna().sum() / len(df) > 0.1:
    issues.append(f"Много пропусков в total_area: {df['total_area'].isna().sum()/len(df)*100:.1f}%")

if issues:
    for i, issue in enumerate(issues, 1):
        print(f"   {i}. {issue}")
else:
    print("   Не найдено критических проблем")

print("\n" + "=" * 60)


## 8. Выводы и рекомендации

### Основные находки (из анализа):
1. **Размер датасета**: 4638 записей, 61 колонка
2. **Пропуски**: 
   - views_count: 100% пропусков (не парсится)
   - developer_*: ~58% пропусков
   - build_year, repair_type, material_type: 18-26% пропусков
3. **Выбросы**: 
   - Цена: 653 выброса (14.1%), max=805 млн руб - критично!
   - total_area: 435 выбросов (9.4%)
4. **Временное разделение**: OK - данные с 2009 по 2026, train/test разделены корректно

### Критические проблемы:
1. **Выбросы в цене**: максимальная цена 805 млн руб - явно ошибка парсинга
2. **deal_type='sale'**: 48 записей с продажей вместо аренды - нужно фильтровать
3. **Пропуски**: views_count не парсится (100% пропусков)

### Рекомендации для подготовки данных:
1. **Очистка данных**:
   - Фильтровать deal_type='sale' (оставить только 'rent')
   - Удалить выбросы в цене: > 10 млн или < 1000 руб
   - Удалить выбросы в площади: > 500 м² или < 10 м²
   - Использовать актуальную цену из price_changes если доступна
   
2. **Feature Engineering**:
   - Добавить цену за м² (price_actual / total_area)
   - Вычислить возраст дома (2025 - build_year)
   - Добавить geo-фичи (расстояние до центра, метро) - уже есть в geo_features.py
   
3. **Разделение данных**:
   - Использовать временное разделение (последние 30 дней = test) - проверено OK
   
4. **Моделирование**:
   - Логарифмировать цену (распределение логнормальное, видны выбросы)
   - CatBoost/LightGBM подходят для работы с пропусками
   - Рассмотреть квантильную регрессию для P10/P50/P90
